In [12]:
import pandas as pd

data = df = pd.read_parquet('/root/autodl-tmp/WSDM/input/train.parquet')

In [13]:
data

,id,prompt,response_a,response_b,winner,model_a,model_b,language
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English
...,...,...,...,...,...,...,...,...
48434,fff5962462043091a771018e37ece11c425e9c7cca64b2...,你好，请你介绍下你自己,你好！我是LSK，一个形式为人工智能的chatbot。,### 介绍\n\n我是一款人工智能语言模型，基于深度学习技术开发而成。我的主要功能是理解自...,model_b,llama-3.2-3b-instruct,llama-3.1-8b-instruct,Chinese
48435,fffad6c95fb82e3a694b4952965852130285e0a38905d5...,Viva Technology参展费用,Viva Technology是一个国际性的科技展览会，每年在法国巴黎举行。参展费用会根据多...,我无法找到Viva Technology参展费用的具体公开信息。\n\n**参展费用通常取决...,model_a,reka-core-20240904,gemini-1.5-flash-exp-0827,English
48436,fffbf502f2cd88b706a4bd6e54ed90e9a2d59d1643140d...,다음은 '사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기'라는 제목의 ...,"사무장병원 경찰 조사 동행기 - 장효강 변호사의 현장 이야기\n\n**""압수수색, ...","도입부를 개선하기 위해서는 독자의 관심을 더 끌어야 하며, 좀 더 명료하고 간결하게...",model_a,gemini-1.5-pro-002,yi-lightning-lite,Korean
48437,fffd2ffdaa03e9e0a0cd1e8f2ee80f530bb19b08fa4312...,What is the height of the church located in Sa...,**Church Information: Église de Saint-Nazaire-...,I couldn't find any information about a church...,model_a,llama-3.1-70b-instruct,llama-3.2-3b-instruct,English


In [2]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    data.loc[val_idx, 'fold'] = int(fold)
data['fold'] = data['fold'].astype(int)

In [3]:
data.to_csv('/root/autodl-tmp/WSDM/input/train.csv', index=False)

In [1]:
import pandas as pd

data = pd.read_csv('/root/autodl-tmp/WSDM/submission.csv')
data2 = pd.read_parquet('/root/autodl-tmp/WSDM/hf-open-models-v1.parquet')

In [2]:
data = pd.merge(data, data2, on='id', how='left')

In [3]:
import numpy as np
filter = data[(data['model_a_prob'] > 0.5) | (data['model_b_prob'] > 0.5)]
filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')

/tmp/ipykernel_56494/4221851404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter['winner'] = np.where(filter['model_a_prob'] > filter['model_b_prob'], 'model_a', 'model_b')


In [4]:
filter[['id', 'prompt',	'response_a','response_b','winner',	'model_a',	'model_b',	'language']].to_csv('/root/autodl-tmp/WSDM/input/extra.csv', index=False)

In [5]:
filter.shape

(8417, 10)

: 